In [1]:
import nltk 
from nltk.stem.lancaster import LancasterStemmer 
stemmer = LancasterStemmer()
from nltk.corpus import stopwords
import numpy as np 
import random 
import json

In [2]:
# import our chat-bot intents file
import json
with open('../input/intents-lol/intents.json') as json_data:
    data = json.load(json_data)
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am 18 years old!', '18 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."], 'context_set': ''}, {'tag': 'shop', 'patterns': ['Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'], 'responses': ['We sell chocolate chip cookies for $2!', 'Cookies ar

In [3]:
words=[]
docs_x=[]
docs_y=[]
labels=[]

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds=nltk.word_tokenize(pattern)
        wrds=[stemmer.stem(w.lower()) for w in wrds if w not in ['?'] ]
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])
        if intent['tag'] not in labels :
            labels.append(intent['tag'])
print(docs_x)
print(docs_y)
        

[['hi'], ['how', 'ar', 'you'], ['is', 'anyon', 'ther'], ['hello'], ['good', 'day'], ['what', 'up'], ['cya'], ['see', 'you', 'lat'], ['goodby'], ['i', 'am', 'leav'], ['hav', 'a', 'good', 'day'], ['how', 'old'], ['how', 'old', 'is', 'tim'], ['what', 'is', 'yo', 'ag'], ['how', 'old', 'ar', 'you'], ['ag'], ['what', 'is', 'yo', 'nam'], ['what', 'should', 'i', 'cal', 'you'], ['what', 'yo', 'nam'], ['id', 'lik', 'to', 'buy', 'someth'], ['what', 'on', 'the', 'menu'], ['what', 'do', 'you', 'reccommend'], ['could', 'i', 'get', 'someth', 'to', 'eat'], ['when', 'ar', 'you', 'guy', 'op'], ['what', 'ar', 'yo', 'hour'], ['hour', 'of', 'op']]
['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'age', 'age', 'age', 'age', 'age', 'name', 'name', 'name', 'shop', 'shop', 'shop', 'shop', 'hours', 'hours', 'hours']


In [4]:
words=sorted(list(set(words)))
labels = sorted(labels)
print(words)
print(labels)
training = []
output =[] 
output_empty = [0 for _ in range ( len(labels))]

['a', 'ag', 'am', 'anyon', 'ar', 'buy', 'cal', 'could', 'cya', 'day', 'do', 'eat', 'get', 'good', 'goodby', 'guy', 'hav', 'hello', 'hi', 'hour', 'how', 'i', 'id', 'is', 'lat', 'leav', 'lik', 'menu', 'nam', 'of', 'old', 'on', 'op', 'reccommend', 'see', 'should', 'someth', 'the', 'ther', 'tim', 'to', 'up', 'what', 'when', 'yo', 'you']
['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']


In [5]:
# reset underlying graph data
bag_of_words=[]
print(docs_x)
print(words)

for docs in docs_x:
    bag=[]
    for w in words : 
        if w in docs :
            bag.append(1)
        else :
            bag.append(0)
    bag_of_words.append(bag)
train_x = np.array(bag_of_words)
print(train_x)

[['hi'], ['how', 'ar', 'you'], ['is', 'anyon', 'ther'], ['hello'], ['good', 'day'], ['what', 'up'], ['cya'], ['see', 'you', 'lat'], ['goodby'], ['i', 'am', 'leav'], ['hav', 'a', 'good', 'day'], ['how', 'old'], ['how', 'old', 'is', 'tim'], ['what', 'is', 'yo', 'ag'], ['how', 'old', 'ar', 'you'], ['ag'], ['what', 'is', 'yo', 'nam'], ['what', 'should', 'i', 'cal', 'you'], ['what', 'yo', 'nam'], ['id', 'lik', 'to', 'buy', 'someth'], ['what', 'on', 'the', 'menu'], ['what', 'do', 'you', 'reccommend'], ['could', 'i', 'get', 'someth', 'to', 'eat'], ['when', 'ar', 'you', 'guy', 'op'], ['what', 'ar', 'yo', 'hour'], ['hour', 'of', 'op']]
['a', 'ag', 'am', 'anyon', 'ar', 'buy', 'cal', 'could', 'cya', 'day', 'do', 'eat', 'get', 'good', 'goodby', 'guy', 'hav', 'hello', 'hi', 'hour', 'how', 'i', 'id', 'is', 'lat', 'leav', 'lik', 'menu', 'nam', 'of', 'old', 'on', 'op', 'reccommend', 'see', 'should', 'someth', 'the', 'ther', 'tim', 'to', 'up', 'what', 'when', 'yo', 'you']
[[0 0 0 ... 0 0 0]
 [0 0 0 ...

In [6]:
train_x
train_x.shape

(26, 46)

In [7]:
output_y=[]
for docs in docs_y :
    output_bag=[]
    for label in labels :
        if docs == label :
            output_bag.append(1)
        else :
            output_bag.append(0)
    output_y.append(output_bag)

train_y=np.array(output_y)
print(train_y)

[[0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 1 0 0]
 [0 0 0 1 0 0]
 [0 0 0 1 0 0]]


In [8]:
import tflearn 
import tensorflow as tf 
# restore all of our data structures
tf.reset_default_graph()
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=500, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 1999  | total loss: 1.23594 | time: 0.015s
| Adam | epoch: 500 | loss: 1.23594 - acc: 0.8738 -- iter: 24/26
Training Step: 2000  | total loss: 1.11839 | time: 0.019s
| Adam | epoch: 500 | loss: 1.11839 - acc: 0.8864 -- iter: 26/26
--
INFO:tensorflow:/kaggle/working/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [9]:
def bag_of_words(s,words):
    bag=[0 for _ in range(len(words))]
    s_words=nltk.word_tokenize(s)
    s_words=[stemmer.stem(w.lower()) for w in s_words if w not in ['?']]
    for se in s_words :
        for i,w in enumerate(words):
            if se == w :
                bag[i]=1
    return np.array(bag)

In [10]:
def chat():
    print("Start talking with the bot (Enter quit to exit): ")
    while True : 
        inp = input ("You : ")
        if inp.lower()== "quit" :
            break
        result = model.predict([bag_of_words(inp,words)])
        index = np.argmax(result)
        tag = labels [index]
        for tg in data['intents']:
            if tg['tag']== tag:
                responses = tg['responses']
        print(random.choice(responses))

In [11]:
chat()

Start talking with the bot (Enter quit to exit): 


You :  hi 


Hi there, how can I help?


You :  what are your hours 


We are open 7am-4pm Monday-Friday!


You :  what do you sell 


We sell chocolate chip cookies for $2!


You :  quit
